In [1]:
import numpy as np
import pandas as pd
import os, random, shutil, sys, time
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import sys

#mac
util_path = "/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/code/latest/util"

#constance
'''args = sys.argv[0:]
py_file = args[0]

code_location = os.path.dirname(os.path.abspath(py_file))
index_of_latest = code_location.index('latest')
util_path = os.path.join(code_location[:index_of_latest], "latest", "util")
'''

sys.path.insert(0, util_path)

In [2]:
starting_dir = os.getcwd()
print (starting_dir)

/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/code/latest/2_bmd/2_bmd_cal_itself/latest/G14


In [3]:
# G14
#complete_file_path = '/Users/kimd999/1_simple.csv'
complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/G14/input/edited/G14_DataForBMC_edited.csv'
df = pd.read_csv(complete_file_path, header = 0)
pd.set_option('display.max_columns', None)
display(df.head())

,chemical.id,dose,ori_num_affected,num_embryos,num_affected
0,Benzophenone,0.00,11.11,36,4
1,Benzophenone,1.00,25.00,36,9
2,Benzophenone,2.54,5.56,36,2
3,Benzophenone,6.45,0.00,36,0
4,Benzophenone,16.40,5.56,36,2


In [4]:
import generate_dose_response as gdr
import BMD_BMDL_estimation as bmdest
import Plot_Save as ps

In [7]:
start_time = time.time()

os.chdir(starting_dir)

if (os.path.isdir("output") == True):
    shutil.rmtree("output")
os.mkdir("output")

output_folder = os.path.join(starting_dir, "output")
os.chdir(output_folder)

chemical_id_from_here = np.unique(df['chemical.id'])

full_devel = "full"
#full_devel = "devel"

if (full_devel == "devel"):
    choose_this_number = min(len(chemical_id_from_here), 1)
    randomly_chosen = random.sample(set(chemical_id_from_here), choose_this_number)
    chemical_id_from_here = []
    for i in range(len(randomly_chosen)):
        chemical_id_from_here.append(str(randomly_chosen[i]))

#chemical_id_from_here = ['Benzophenone']
#chemical_id_from_here = ['Benzyl butyl phthalate']
#chemical_id_from_here = ['Galaxolide']

total_number_of_chemicals_to_processed = len(chemical_id_from_here)
number_of_chemicals_processed = 0

for chemical_id in chemical_id_from_here:
    print("\nchemical_id:" + str(chemical_id))
    os.chdir(output_folder)
    # subset original dataframe for a user-specified chemical
    df_per_chemical = df.loc[df['chemical.id'] == chemical_id,['chemical.id', 'dose', 'num_embryos', 'num_affected']]
    
    # Fit dose response models
    df_per_chemical = df_per_chemical.sort_values(by=['dose'])
    #display(df_per_chemical)
        
    test_dose_response = gdr.reformat_dose_response(df_per_chemical)
    # cp num_embryos -> total_num
    display(test_dose_response.head())
    
    bmd_feasibility_flag = gdr.BMD_feasibility_analysis(test_dose_response)
    print ("bmd_feasibility_flag:"+str(bmd_feasibility_flag))
    
    if (bmd_feasibility_flag == 1):
        continue
        
    model_predictions = bmdest.analyze_dose_response_data(test_dose_response)
    #print (model_predictions)
    
    # Select best model
    selected_model_params = bmdest.select_model(model_predictions)
    print (selected_model_params)

    # Check if unique model is found
    unique_model_flag = selected_model_params['no_unique_model_found_flag']

    if(unique_model_flag == 0):
        ps.save_results_good_data_unique_model(test_dose_response, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), "end_point")
    else:
        bmd_analysis_flag = selected_model_params['model_select_flag']
        if(bmd_analysis_flag == 1):
            ps.save_results_poor_data_or_no_convergence(test_dose_response, bmd_feasibility_flag, str(chemical_id), "end_point", selected_model_params)
        else:
            ps.save_results_good_data_nounique_model(test_dose_response, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), "end_point")
            
    number_of_chemicals_processed += 1
    print_this = str(number_of_chemicals_processed) + " chemicals processed out of " + str(total_number_of_chemicals_to_processed)
    print(print_this)
        
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("BMD calculation is done, it took:"+str(time_took))
# took 445 seconds for all ~13 chemicals


chemical_id:Benzophenone


,dose,num_affected,total_num,num_embryos
0,0.00,4,36,36
1,1.00,9,36,36
2,2.54,2,36,36
3,6.45,0,36,36
4,16.40,2,36,36


dose_response:
     dose  num_affected  total_num  num_embryos
0    0.00             4         36           36
1    1.00             9         36           36
2    2.54             2         36           36
3    6.45             0         36           36
4   16.40             2         36           36
5   35.00             7         36           36
6   74.80            22         36           36
7  100.00            24         35           35
dose_response['num_affected']:
0     4
1     9
2     2
3     0
4     2
5     7
6    22
7    24
Name: num_affected, dtype: int64
dose_response['num_embryos']:
0    36
1    36
2    36
3    36
4    36
5    36
6    36
7    35
Name: num_embryos, dtype: int64
frac_response:
0    0.111111
1    0.250000
2    0.055556
3    0.000000
4    0.055556
5    0.194444
6    0.611111
7    0.685714
dtype: float64
frac_response[0]:
0.1111111111111111
data_corr:SpearmanrResult(correlation=0.5030030300035688, pvalue=0.203876682008432)
data_corr[0]:0.5030030300035688
true

,dose,num_affected,total_num,num_embryos
0,0.00,1,36,36
1,2.25,0,36,36
2,3.00,4,36,36
3,3.50,5,36,36
4,4.00,14,36,36


dose_response:
   dose  num_affected  total_num  num_embryos
0  0.00             1         36           36
1  2.25             0         36           36
2  3.00             4         36           36
3  3.50             5         36           36
4  4.00            14         36           36
5  4.50            21         36           36
6  5.00            29         36           36
7  8.00            36         36           36
dose_response['num_affected']:
0     1
1     0
2     4
3     5
4    14
5    21
6    29
7    36
Name: num_affected, dtype: int64
dose_response['num_embryos']:
0    36
1    36
2    36
3    36
4    36
5    36
6    36
7    36
Name: num_embryos, dtype: int64
frac_response:
0    0.027778
1    0.000000
2    0.111111
3    0.138889
4    0.388889
5    0.583333
6    0.805556
7    1.000000
dtype: float64
frac_response[0]:
0.027777777777777776
data_corr:SpearmanrResult(correlation=0.9761904761904763, pvalue=3.3143960262001043e-05)
data_corr[0]:0.9761904761904763
true bmd_feasib

...

dose_response:
    dose  num_affected  total_num  num_embryos
0    0.0             1         36           36
1   10.0             4         36           36
2   30.0             1         36           36
3   50.0             7         36           36
4   60.0            14         36           36
5   70.0            17         36           36
6   80.0            23         36           36
7  100.0            32         36           36
dose_response['num_affected']:
0     1
1     4
2     1
3     7
4    14
5    17
6    23
7    32
Name: num_affected, dtype: int64
dose_response['num_embryos']:
0    36
1    36
2    36
3    36
4    36
5    36
6    36
7    36
Name: num_embryos, dtype: int64
frac_response:
0    0.027778
1    0.111111
2    0.027778
3    0.194444
4    0.388889
5    0.472222
6    0.638889
7    0.888889
dtype: float64
frac_response[0]:
0.027777777777777776
data_corr:SpearmanrResult(correlation=0.9581010095306072, pvalue=0.0001781567727650473)
data_corr[0]:0.9581010095306072
true bm

,dose,num_affected,total_num,num_embryos
0,0.0,0,36,36
1,2.0,1,36,36
2,3.0,0,36,36
3,4.0,3,36,36
4,5.0,7,36,36


dose_response:
   dose  num_affected  total_num  num_embryos
0   0.0             0         36           36
1   2.0             1         36           36
2   3.0             0         36           36
3   4.0             3         36           36
4   5.0             7         36           36
5   6.0            17         36           36
6   8.0            24         36           36
7  10.0            31         36           36
dose_response['num_affected']:
0     0
1     1
2     0
3     3
4     7
5    17
6    24
7    31
Name: num_affected, dtype: int64
dose_response['num_embryos']:
0    36
1    36
2    36
3    36
4    36
5    36
6    36
7    36
Name: num_embryos, dtype: int64
frac_response:
0    0.000000
1    0.027778
2    0.000000
3    0.083333
4    0.194444
5    0.472222
6    0.666667
7    0.861111
dtype: float64
frac_response[0]:
0.0
data_corr:SpearmanrResult(correlation=0.9581010095306072, pvalue=0.0001781567727650473)
data_corr[0]:0.9581010095306072
true bmd_feasibility_flag:2
{'mod

,dose,num_affected,total_num,num_embryos
0,0.0,2,36,36
1,2.0,2,36,36
2,3.0,3,36,36
3,4.0,8,36,36
4,4.5,19,36,36


dose_response:
   dose  num_affected  total_num  num_embryos
0   0.0             2         36           36
1   2.0             2         36           36
2   3.0             3         36           36
3   4.0             8         36           36
4   4.5            19         36           36
5   5.0            21         36           36
6   8.0            36         36           36
7  10.0            36         36           36
dose_response['num_affected']:
0     2
1     2
2     3
3     8
4    19
5    21
6    36
7    36
Name: num_affected, dtype: int64
dose_response['num_embryos']:
0    36
1    36
2    36
3    36
4    36
5    36
6    36
7    36
Name: num_embryos, dtype: int64
frac_response:
0    0.055556
1    0.055556
2    0.083333
3    0.222222
4    0.527778
5    0.583333
6    1.000000
7    1.000000
dtype: float64
frac_response[0]:
0.05555555555555555
data_corr:SpearmanrResult(correlation=0.9880235200593538, pvalue=4.256164416841794e-06)
data_corr[0]:0.9880235200593538
true bmd_feasibil